In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import string # for milestone 1
from collections import Counter

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2025-sep-dl-gen-ai-project/sample_submission.csv
/kaggle/input/2025-sep-dl-gen-ai-project/train.csv
/kaggle/input/2025-sep-dl-gen-ai-project/test.csv


# milestone 2   

In [2]:
train_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
test_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')

In [3]:
emotion_labels = ['anger', 'fear', 'joy', 'sadness', 'surprise']

### TfidfVectorizer 

In [4]:
X_train = train_df['text']
Y_train = train_df[emotion_labels]

X_test = test_df['text']
test_ids = test_df['id']

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

In [7]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"feature matrix shape (train): {X_train_tfidf.shape}")
print(f"feature matrix shape (test): {X_test_tfidf.shape}")

feature matrix shape (train): (6827, 33896)
feature matrix shape (test): (1707, 33896)


### DTC

In [37]:
df_train_full = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
df_test_final = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')

df_train_full['text'] = df_train_full['text'].fillna('')
df_test_final['text'] = df_test_final['text'].fillna('')

emotion_labels = ['anger', 'fear', 'joy', 'sadness', 'surprise']
test_ids = test_df['id']

y = df_train_full[emotion_labels]

X_text = df_train_full['text']

In [38]:
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

In [39]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

X_train_tfidf = vectorizer.fit_transform(X_train_text)

X_val_tfidf = vectorizer.transform(X_val_text)

model = DecisionTreeClassifier(random_state=42, max_depth=6)

model.fit(X_train_tfidf, y_train) 

DecisionTreeClassifier(max_depth=6, random_state=42)

In [42]:
total_nodes = model.tree_.node_count
print(f"Nodes created by the model (single tree): {total_nodes}")

y_pred_train = model.predict(X_train_tfidf)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
print(f"Macro F1 score on training split: {train_f1:.4f}")

y_pred_val = model.predict(X_val_tfidf)
val_f1 = f1_score(y_val, y_pred_val, average='macro')
print(f"Macro F1 score on test split: {val_f1:.4f}")

Nodes created by the model (single tree): 55
Macro F1 score on training split: 0.2146
Macro F1 score on test split: 0.1970


In [44]:
X_test_final_tfidf = vectorizer.transform(df_test_final['text'])
y_pred_final = model.predict(X_test_final_tfidf)

df_submission = pd.DataFrame(y_pred_final, columns=emotion_labels)

df_submission.insert(0, 'id', test_ids)

for col in emotion_labels:
    df_submission[col] = df_submission[col].astype(int)

df_submission.to_csv("submission.csv", index=False)

print(f"Submission file created")

Submission file created
